# Overview

Let $\sigma_{\Gamma}\in \Sigma_{\Gamma}^{d}$ (where $d=b_{1}(\Gamma)$) and define $f_{\sigma_{\Gamma}}$ to be the map

$$\Gamma^{'} \mapsto \operatorname{Min}\left\{\sum_{v \in \Gamma^{'}} \vec{d} \mid \vec{d} \in \sigma_{\Gamma}(\Gamma)\right\}$$

where $\Gamma^{'}$ is an elementary induced subgraph of $\Gamma$ (completely determined by its vertices).

Let $\Gamma$ be the graph $F$ described in the thesis (Filippo Viviani graph) and let $D$ be the set of  $\sigma_{\Gamma}\in \Sigma_{\Gamma,T,D_T}$ (for $D_T=\vec{0}$ and $T$ I have fixed earlier) which are not induced by a $\phi \in V^{d}(\Gamma)$ (16 in total).

In this notebook we return $f_{\sigma_{\Gamma}}$ for a $\sigma_{\Gamma}\in D$ as stated in the thesis. We use $f_{\sigma_{\Gamma}}$ to give the example in Section 4.2.

We show that $f_{\sigma_{\Gamma}}(\Gamma^{'})+f_{\sigma_{\Gamma}}(\Gamma^{'c}) = d- |\#E(\Gamma) \setminus (E(\Gamma^{'} \cup E(\Gamma^{'c})|+1$ for all elementtary induced subgraphs $\Gamma^{'}$.

# Table of contents

1. [Functions](#s1)
2. [$f_{\sigma_{\Gamma}}$ for $\sigma_{\Gamma} \in D$](#s2)


# Functions <a name="s1"></a>

In [1]:
import pickle
import numpy as np

In [2]:
def get_first_betti_number(G):
    #for if the degree of divisors on trees is 0, the degree of lbm is this.
    return len(G.edges())-len(G.vertices())+1

The next functions allows one to define $f_{\sigma_{\Gamma}}$ by taking the minimum of the sum of $I$ vertices.

In [3]:
def sum_comps(comp,indexor): #Done
    """returns the sum of the components of comp in the position by indexor"""
    #for d and phi
    sum_l=[comp[int(i)] for i in indexor] #Becuase of the vertex labeling of G having v_0....
    return sum(sum_l)

In [4]:
def get_min_val(indexor, sig_lst):
    """
    Calculate the minimum value of sums in a list.

    Input:
    indexor: A string representing positions in the list, e.g., "0243".
    sig_lst: A list of lists, e.g., [0, 6, 2, 4, 6].
    """

    x = set()
    for divisor in sig_lst:
        divisor_comp_sum = sum_comps(divisor, indexor)
        x.add(divisor_comp_sum)

    min_val = min(x)  # Calculate the minimum value in the set

    return min_val

The next function gives all pairs $(\Gamma^{'},\Gamma^{'c})$ corresponding to elementary cuts.

In [5]:
def get_pairs_con_subg(G):# for phi_tree_inequalities_data
    
    """
    Note: we are not working with multigraphs so connected_subgraph_iterator should give subgraphs with all edges between    
    Objective: Used to get cut info and total degree adjustment
    Input:
    Return: Pairs of complete subgraphs for V and V^{c} such that V scup V^{c}=V(G)
    """
    
    vert_G=G.vertices()
    edges_G=G.edges()
    
    #get list of connected subgraphs:
    
    #Want to get pairs of complete subgraphs for pairs of vertices that is a is disjoint union of all verts
    ll=list(G.connected_subgraph_iterator())

    # put them into pairs, where vertices are a disjoint union of G.
    pairs=[]
    for i in ll:
        a=i.vertices()
        for j in ll:
            b=j.vertices()
            if sorted(a+b)==vert_G:
                #May need to take complete graph here containing a, and for b. 
                if (j,i) in pairs: # If we have (\Gamma^{'},\Gamma^{'c}) in pairs we dont need (\Gamma^{'c},\Gamma^{'})
                    continue
                else:
                    pairs.append((i,j))
    return pairs

The next function gives us the cut information with respect to a pair of vertices corresponding to $(\Gamma^{'},\Gamma^{'c})$.

In [7]:
def get_cut_edges(G,pair): #for phi_tree_inequalities_data
    
    """
    Obj:Get list of the edges in a cut for a given pair, 
    for a pair of connected subgraphs that partitions the vertices of G.
    
    Return: list of edges in cut for a pair of connected subgraphs that partition the vertices of G 
    """
    a_edges=pair[0].edges()
    b_edges=pair[1].edges()
    total_edges=a_edges+b_edges
    
    edges_G=G.edges()

    #Cut edges.
    cut_edges=[ e for e in edges_G if e not in total_edges]
    
    return cut_edges

I return $f_{\sigma_{\Gamma}}$ as a dictionary with keys recording the elementary induced subgraphs $(\Gamma^{'},\Gamma^{'c})$ with values recording $f_{\sigma_{\Gamma}}(\Gamma^{'}),f_{\sigma_{\Gamma}}(\Gamma^{'c}),$ the number of edgess in the cut of $(\Gamma^{'},\Gamma^{'c})$ and a True or False statement if the pair satisfies,

$$f_{\sigma_{\Gamma}}(\Gamma^{'})+f_{\sigma_{\Gamma}}(\Gamma^{'c}) = d- |\#E(\Gamma) \setminus (E(\Gamma^{'} \cup E(\Gamma^{'c})|+1.$$

The following checks the above condition.

In [25]:
def degree_statment(val_1,val_2,deg,cut_edges):
    x=f"{val_1}+{val_2}={deg}-{cut_edges}+1"
    if val_1+val_2==deg-cut_edges+1:
        return True,x
    else:
        return False,x

The following gives the values of $f_{\sigma_{\Gamma}}$.

In [9]:
def alpha_sig_gamma(G, sig_lst):
    """
    Objective: Obtains the min of d in sigma{Gamma}(Gamma) for each elementary induced subgraph
    
    Inputs:
    G: graph
    sig_lst: the set sigma{Gamma}(Gamma) as a list of lists of divisors
    
    Returns phi_G as a
    Dict contains the vertices elementary induced subg: key
    and value: min value of d summed over subg
    """

    deg = get_first_betti_number(G)

    pairs = get_pairs_con_subg(G) #elementary induced subgraphs

    phi_G = {} #alpha function
    for pair in pairs:
        
        
        edges_cut=len(get_cut_edges(G,pair))
        
        # Get vertices of subg in pair and turn to "v1v2v3" format.
        subg1_l = pair[0].vertices()
        elem_ind_subg_v1 = ''.join(subg1_l)
        min_val1 = get_min_val(elem_ind_subg_v1, sig_lst)
        
        # Get vertices of subg in pair and turn to "v1v2v3" format.
        subg2_l = pair[1].vertices()
        elem_ind_subg_v2 = ''.join(subg2_l)
        min_val2 = get_min_val(elem_ind_subg_v2, sig_lst)

        
        #Additional want so smallest induce subg is first, change key and value to correct
        if len(elem_ind_subg_v1)>len(elem_ind_subg_v2):
            
            statement=degree_statment(min_val1,min_val2,deg,edges_cut)
            key=(elem_ind_subg_v2,elem_ind_subg_v1)
            
            
#             val=(min_val2,min_val1,edges_cut,statement)
            a='%g'%((edges_cut*0.5)+min_val1-1) #marco
            b='%g'%((edges_cut*0.5)+min_val1)



            e='%g'%((edges_cut*0.5)+min_val2-1)
            f='%g'%((edges_cut*0.5)+min_val2)


#             val=((elem_ind_subg_v2,[(e,f),(g,h)]),(elem_ind_subg_v1,[(a,b),(c,d)]))
            val=((elem_ind_subg_v2,[(e,f),(min_val2,statement)]),(elem_ind_subg_v1,[(a,b),(min_val1,statement)]))
#             val=((elem_ind_subg_v2,min_val2),(elem_ind_subg_v1,min_val1))


            
            # Add key:value pair # {[('0','12345'): (0,1,3,True)]}
            phi_G[key] = val  # Use dictionary notation to add key:value pairs
            continue
        
        
        statement=degree_statment(min_val1,min_val2,deg,edges_cut)
        
        
        key=(elem_ind_subg_v1,elem_ind_subg_v2)
        
        a='%g'%((edges_cut*0.5)+min_val1-1) #marco
        b='%g'%((edges_cut*0.5)+min_val1)
        

        
        e='%g'%((edges_cut*0.5)+min_val2-1)
        f='%g'%((edges_cut*0.5)+min_val2)
        
        
#         val=((elem_ind_subg_v1,[(a,b),(c,d)]),(elem_ind_subg_v2,[(e,f),(g,h)]))
        val=((elem_ind_subg_v1,[(a,b),(min_val1,statement)]),(elem_ind_subg_v2,[(e,f),(min_val2,statement)]))
#         val=((elem_ind_subg_v1,min_val1),(elem_ind_subg_v2,min_val2))

#         val=(min_val1,min_val2,edges_cut,statement)
        
        # Add key:value pair # {[('0','12345'): (0,1,3,True)]}
        phi_G[key] = val  # Use dictionary notation to add key:value pairs

    return phi_G

# $f_{\sigma_{\Gamma}}$ for $\sigma_{\Gamma} \in D$ <a name="s2"></a>

Note the graph $F$ here have vertices starting from 0 to $5$, whereas in the Example in the thesis we have labeled the vertices $1$ to $6$. Therefore it is necessary to map everything accordingly.

In [10]:
graphname="FV_G6M3M14" 
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3"),("1","4")], multiedges=True)

We load the set $D$.

In [20]:
pickle_in = open(r"FV_G6M3M14_degen.pkl", "rb")
# "FV_G6M3M14\FV_G6M3M14_degen.pkl"
data = pickle.load(pickle_in)
pickle_in.close()

Let $\sigma_{\Gamma} \in D $ (we take the first one in the list as an example) and consider $\sigma_{\Gamma}(\Gamma)$. The divisors of this set are of the form $\vec{d}=[d_0v_0,d_1 v_1,d_2 v_2,d_3v_3,d_4v_4,d_5v_5]$

In [21]:
sc_1=data[0]
sc_1g=sc_1[1].tolist() #the set \sigma_{\Gamma}(\Gamma) as a list of lists of divisors
# sc_1g
sc_1gnp=sc_1[1]
# sc_1gnp #for viewing purposes
sc_1g

[[0, 0, 1, 1, 0, 1],
 [0, 1, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 1],
 [0, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, -1, 1],
 [0, 1, 1, 1, 0, 0],
 [0, 2, 0, 0, 0, 1],
 [0, 2, 0, 0, 1, 0],
 [0, 2, 0, 1, -1, 1],
 [0, 2, 0, 1, 0, 0],
 [0, 2, 1, 0, -1, 1],
 [0, 2, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, -1, 1],
 [1, 1, 0, 1, 0, 0],
 [1, 1, 1, -1, 0, 1],
 [1, 1, 1, -1, 1, 0],
 [1, 1, 1, 0, -1, 1],
 [1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, -1, 0],
 [1, 2, 0, 0, -1, 1],
 [1, 2, 0, 0, 0, 0],
 [1, 2, 0, 1, -1, 0],
 [1, 2, 1, -1, -1, 1],
 [1, 2, 1, -1, 0, 0],
 [1, 2, 1, 0, -1, 0],
 [2, 0, 1, -1, 1, 0],
 [2, 0, 1, 0, 0, 0],
 [2, 1, 0, 0, 0, 0],
 [2, 1, 0, 1, -1, 0],
 [2, 1, 1, -1, 0, 0],
 [2, 1, 1, 0, -1, 0]]

In [17]:
# # marco likes it translated for ease
# sc_1g=[list(np.array(x)+np.array([0,0,0,1,1,0])) for x in sc_1g]
# sc_1g

For each $\Gamma^{'}_{I}$ (elementary induced subgraph) we consider the region of $\phi_{I}$ wrt the following inequality.

$$f_{\sigma_{\Gamma}}({I})-1+cut_I< \phi_{I} < f_{\sigma_{\Gamma}}({I})+cut_I.$$

In [26]:
# elem_ind_subg
asg=alpha_sig_gamma(G,sc_1g)
asg

l=[]
for i in asg.values():
    print(i,"\n")
    for j in i:
        l.append(j)

(('0', [('0.5', '1.5'), (0, (True, '0+1=3-3+1'))]), ('12345', [('1.5', '2.5'), (1, (True, '0+1=3-3+1'))])) 

(('01', [('1', '2'), (0, (True, '0+0=3-4+1'))]), ('2345', [('1', '2'), (0, (True, '0+0=3-4+1'))])) 

(('012', [('2', '3'), (1, (True, '1+-1=3-4+1'))]), ('345', [('0', '1'), (-1, (True, '1+-1=3-4+1'))])) 

(('45', [('-0.5', '0.5'), (-1, (True, '2+-1=3-3+1'))]), ('0123', [('2.5', '3.5'), (2, (True, '2+-1=3-3+1'))])) 

(('5', [('0', '1'), (0, (True, '2+0=3-2+1'))]), ('01234', [('2', '3'), (2, (True, '2+0=3-2+1'))])) 

(('4', [('-0.5', '0.5'), (-1, (True, '2+-1=3-3+1'))]), ('01235', [('2.5', '3.5'), (2, (True, '2+-1=3-3+1'))])) 

(('3', [('-0.5', '0.5'), (-1, (True, '2+-1=3-3+1'))]), ('01245', [('2.5', '3.5'), (2, (True, '2+-1=3-3+1'))])) 

(('34', [('-1', '0'), (-2, (True, '2+-2=3-4+1'))]), ('0125', [('3', '4'), (2, (True, '2+-2=3-4+1'))])) 

(('2', [('0', '1'), (0, (True, '2+0=3-2+1'))]), ('01345', [('2', '3'), (2, (True, '2+0=3-2+1'))])) 

(('23', [('0.5', '1.5'), (0, (True, '1+0

For example:('0', [('0.5', '1.5'), (0, (True, '0+1=3-3+1'))]) means that.

On $\Gamma_{I}^{'}$ with $I=\{v_{0}\}$ we have that,

$$0.5< \phi_{I} < 1.5.$$

that $f_{\sigma_{\Gamma}}({I})=0$ and 

$$f_{\sigma_{\Gamma}}({I})-1+cut_I< \phi_{I} < f_{\sigma_{\Gamma}}({I})+cut_I.$$

and for this case we have:

$$f_{\sigma_{\Gamma}}(\Gamma^{'})+f_{\sigma_{\Gamma}}(\Gamma^{'c}) = d- |\#E(\Gamma) \setminus (E(\Gamma^{'} \cup E(\Gamma^{'c})|+1.$$

is equivelent to $0+1=3-3+1$.

# On genus $1$ subgraphs

In this section I would like to determine the genus $1$ stability conditions which I conjecture determine the stability condition. In particular I want to see that all other cycles are determined by identification and restriction.

In [ ]:
As always we consider the same stability condition which fails to be induced by a polarisation.

In [ ]:
pickle_in = open(r"FV_G6M3M14_degen.pkl", "rb")
# "FV_G6M3M14\FV_G6M3M14_degen.pkl"
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
sc_1=data[0]
sc_1g=sc_1[1].tolist() #the set \sigma_{\Gamma}(\Gamma) as a list of lists of divisors
# sc_1g

In [ ]:
sc_1gnp=sc_1[1]
# sc_1gnp #for viewing purposes
sc_1g

Which maximal spanning subgraph are we on and what are the cycles?

Subgraph 6: [('0', '1', 'e1'), ('0', '5', 'e6'), ('1', '2', 'e2'), ('2', '3', 'e3'), ('3', '4', 'e4'), ('4', '5', 'e5')]
((1, 4, 6, 2, 3, 5), [('0', '5', None), ('1', '2', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
Tree 2
assignment [0, 0, 1, -1, 0, 0]

Subgraph 15: [('0', '3', 'e7'), ('0', '5', 'e6'), ('1', '2', 'e2'), ('1', '4', 'e8'), ('2', '3', 'e3'), ('4', '5', 'e5')]
((7, 8, 3, 6, 5, 2), [('0', '5', None), ('1', '2', None), ('1', '4', None), ('2', '3', None), ('4', '5', None)]),
tree 4
assignment [0, 0, 1, -1, 0, 0]

[[0, 0, 0, 0, 0, 0], [0, 0, 1, -1, 0, 0], [0, 1, 0, -1, 0, 0], [0, 0, 1, -1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, -1, 0], [0, 1, 0, 0, -1, 0], [0, 1, 0, 0, -1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 1, -1, -1, 0], [0, 0, 1, 0, -1, 0], [1, 1, 0, -1, -1, 0], [0, 1, 0, 0, -1, 0], [0, 1, 0, 0, -1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, -1, 0], [0, 1, 0, -1, 0, 0], [0, 1, 1, -1, -1, 0], [0, 0, 0, 0, -1, 1], [0, 1, 0, 0, -1, 0], [0, 1, 0, -1, -1, 1], [1, 0, 1, -1, -1, 0], [0, 1, 1, -1, -1, 0], [0, 2, 0, -1, -1, 0], [0, 1, 1, -1, -1, 0], [0, 0, 0, 1, -1, 0], [0, 1, 0, 0, -1, 0], [0, 1, 0, 0, -1, 0], [1, 1, 0, -1, -1, 0], [1, 0, 0, 0, -1, 0], [1, 0, 0, 0, -1, 0], [0, 1, 0, 0, -1, 0], [1, 1, 0, -1, -1, 0], [1, 1, 0, -1, -1, 0], [1, 1, 0, -1, -1, 0]]

[[0, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 1, 0], [0, 1, 1, 1, -1, 1], [0, 1, 1, 1, 0, 0], [0, 2, 0, 0, 0, 1], [0, 2, 0, 0, 1, 0], [0, 2, 0, 1, -1, 1], [0, 2, 0, 1, 0, 0], [0, 2, 1, 0, -1, 1], [0, 2, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 1], [1, 1, 0, 0, 1, 0], [1, 1, 0, 1, -1, 1], [1, 1, 0, 1, 0, 0], [1, 1, 1, -1, 0, 1], [1, 1, 1, -1, 1, 0], [1, 1, 1, 0, -1, 1], [1, 1, 1, 0, 0, 0], [1, 1, 1, 1, -1, 0], [1, 2, 0, 0, -1, 1], [1, 2, 0, 0, 0, 0], [1, 2, 0, 1, -1, 0], [1, 2, 1, -1, -1, 1], [1, 2, 1, -1, 0, 0], [1, 2, 1, 0, -1, 0], [2, 0, 1, -1, 1, 0], [2, 0, 1, 0, 0, 0], [2, 1, 0, 0, 0, 0], [2, 1, 0, 1, -1, 0], [2, 1, 1, -1, 0, 0], [2, 1, 1, 0, -1, 0]]

[((1, 4, 7, 8), [('0', '3', None), ('1', '2', None), ('1', '4', None), ('3', '4', None), ('4', '5', None)])
, ((7, 5, 4, 6), [('0', '1', None), ('0', '5', None), ('1', '2', None), ('3', '4', None), ('4', '5', None)])
, ((1, 7, 2, 3), [('0', '3', None), ('1', '2', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((1, 6, 8, 5), [('0', '5', None), ('1', '2', None), ('1', '4', None), ('3', '4', None), ('4', '5', None)]),
((2, 3, 4, 8), [('0', '1', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((1, 4, 6, 2, 3, 5), [('0', '5', None), ('1', '2', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((7, 4, 6, 5), [('0', '5', None), ('1', '2', None), ('1', '4', None), ('3', '4', None), ('4', '5', None)]),
((2, 4, 3, 8), [('0', '3', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((7, 4, 6, 5), [('0', '5', None), ('1', '2', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((2, 3, 8, 4), [('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((1, 6, 5, 8), [('0', '3', None), ('0', '5', None), ('1', '2', None), ('1', '4', None), ('4', '5', None)]),
((1, 3, 7, 2), [('0', '3', None), ('1', '2', None), ('1', '4', None), ('2', '3', None), ('4', '5', None)]),
((1, 7, 2, 3), [('0', '3', None), ('0', '5', None), ('1', '2', None), ('2', '3', None), ('4', '5', None)]),
((1, 8, 6, 5), [('0', '5', None), ('1', '2', None), ('1', '4', None), ('2', '3', None), ('4', '5', None)]),
((7, 8, 3, 6, 5, 2), [('0', '5', None), ('1', '2', None), ('1', '4', None), ('2', '3', None), ('4', '5', None)]),
((1, 7, 8, 4), [('0', '3', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((7, 5, 4, 6), [('0', '1', None), ('0', '5', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((1, 6, 8, 5), [('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]),
((7, 6, 5, 4), [('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None), ('4', '5', None)]), 
((1, 8, 6, 5), [('0', '3', None), ('0', '5', None), ('1', '4', None), ('2', '3', None), ('4', '5', None)]),
((1, 4, 8, 7), [('0', '3', None), ('0', '5', None), ('1', '2', None), ('1', '4', None), ('3', '4', None)]),
((1, 2, 3, 7), [('0', '3', None), ('0', '5', None), ('1', '2', None), ('2', '3', None), ('3', '4', None)]),
((2, 3, 8, 4), [('0', '1', None), ('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None)]),
((2, 4, 3, 8), [('0', '3', None), ('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None)]),
((1, 3, 7, 2), [('0', '3', None), ('0', '5', None), ('1', '2', None), ('1', '4', None), ('2', '3', None)]),
((1, 8, 7, 4), [('0', '3', None), ('0', '5', None), ('1', '4', None), ('2', '3', None), ('3', '4', None)])]

# Leftovers

## Counterexample

In [ ]:
determine a counterexample to show fails existence of phi

In [19]:
def summer(data,columns_to_sum,val):
    # Use numpy.sum along axis=1 to sum the specified columns for each row
    sums = np.sum(data[:, columns_to_sum], axis=1)

    result_list = []

    for i, s in enumerate(sums):
        if s == val:
            result_list.append(s)
        else:
            result_list.append(None)

    for i,t in enumerate(result_list):
        if t==None:
            continue
        print(f"{columns_to_sum}::{i+1}: {t}")

In [20]:
# Assuming your array is named 'data'
data = sc_1gnp
#consider values -1,3 only

In [27]:
# for x in [[2,3,4,5],[1,2,4,5],[1,2,4],[2,3,4]]:
#     columns_to_sum = x # not in v_1,..,v_6 format but v_0,..,v_5 format
#     val=1
#     summer(data,columns_to_sum,val)
#     print("---")


In [28]:
# for x in [[0,1,2],[0,1,5],[0,2,3],[0,3,5],[0,4,5],[3,4,5],[2,3,4],[1,4,5],[1,2,5],[1,2,3]]:
#     columns_to_sum = x # not in v_1,..,v_6 format but v_0,..,v_5 format
#     val=3
#     summer(data,columns_to_sum,val)
#     print("---")


In [29]:
# for x in [[0,1],[0,3],[3,4],[1,4]]:
#     columns_to_sum = x # not in v_1,..,v_6 format but v_0,..,v_5 format
#     val=-1
#     summer(data,columns_to_sum,val)
#     print("---")


In [30]:
# for x in [[0,1],[0,3],[3,4],[1,4]]:
#     columns_to_sum = x # not in v_1,..,v_6 format but v_0,..,v_5 format
#     val=3
#     summer(data,columns_to_sum,val)
#     print("---")


In [31]:
# for x in [[0,2,3,5],[1,2,4,5],[0,1,2,5],[2,3,4,5]]:
#     columns_to_sum = x # not in v_1,..,v_6 format but v_0,..,v_5 format
#     val=3
#     summer(data,columns_to_sum,val)
#     print("---")


In [32]:
# # Specify the indices of the columns you want to sum (1, 3, 4 in this case)

# columns_to_sum = [1,2,4] # not in v_1,..,v_6 format but v_0,..,v_5 format
# val=3
# summer(data,columns_to_sum,val)
# print("---")

# columns_to_sum = [2,3,4] # not in v_1,..,v_6 format but v_0,..,v_5 format
# val=-1
# summer(data,columns_to_sum,val)
# print("---")


In [122]:
columns_to_sum = [2,3,4] # not in v_1,..,v_6 format but v_0,..,v_5 format
val=3
summer(data,columns_to_sum,val)
print("---")


columns_to_sum = [1,4,5] # not in v_1,..,v_6 format but v_0,..,v_5 format
val=-1
summer(data,columns_to_sum,val)
print("---")

---
---


## leftovers

In [59]:
data=l

In [125]:
def get_pairs_with_one_difference_and_specific_digits(data, digit1, digit2):
    # Extract tuples with strings that are different by only one term and contain specific digits
    pair1 = None
    pair2 = None

    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            str1, str2 = data[i][0], data[j][0]
            val1, val2 = data[i][1], data[j][1]
            if (
                len(str1) == len(str2)
                and val1 != val2
                and sum(1 for a, b in zip(str1, str2) if a != b) == 1
                and digit1 in str1 and digit2 in str2
            ):
                intersection_set = set(str1) & set(str2)
                difference_set = set([digit1, digit2])
                
                # Exclude pairs where intersection and difference have a common element
                if not intersection_set & difference_set:
                    if pair1 is None:
                        pair1 = {'pair': [data[i], data[j]], 'difference': difference_set, 'intersection': intersection_set}
                    elif pair2 is None:
                        pair2 = {'pair': [data[j], data[i]], 'difference': difference_set, 'intersection': intersection_set}
                    else:
                        break

    return pair1, pair2

# Example usage with your data, looking for pairs with '1' in one and '4' in the other
pair1, pair2 = get_pairs_with_one_difference_and_specific_digits(data, '1', '3')
print("Pair 1:", pair1)
print("Pair 2:", pair2)


Pair 1: None
Pair 2: None


In [124]:
digits = [1, 2, 3, 4, 5]

for digit1 in digits:
    for digit2 in digits:
        # Skip cases where digit1 and digit2 are the same
        if digit1 == digit2:
            continue

        # Example usage with your data, looking for pairs with digit1 in one and digit2 in the other
        pair1, pair2 = get_pairs_with_one_difference_and_specific_digits(data, str(digit1), str(digit2))

        # Check if both pairs are not None
        if pair1 is not None and pair2 is not None:
            print(f"Pair 1 for {digit1} and {digit2}:", pair1)
            print(f"Pair 2 for {digit1} and {digit2}:", pair2)
            print("\n")
            


Pair 1 for 1 and 3: {'pair': [('0145', 1), ('0345', 0)], 'difference': {'3', '1'}, 'intersection': {'0', '4', '5'}}
Pair 2 for 1 and 3: {'pair': [('035', 0), ('015', 1)], 'difference': {'3', '1'}, 'intersection': {'0', '5'}}


Pair 1 for 3 and 1: {'pair': [('345', -1), ('145', 0)], 'difference': {'3', '1'}, 'intersection': {'4', '5'}}
Pair 2 for 3 and 1: {'pair': [('1', 0), ('3', -1)], 'difference': {'3', '1'}, 'intersection': set()}


Pair 1 for 4 and 2: {'pair': [('4', -1), ('2', 0)], 'difference': {'2', '4'}, 'intersection': set()}
Pair 2 for 4 and 2: {'pair': [('12', 1), ('14', 0)], 'difference': {'2', '4'}, 'intersection': {'1'}}




In [107]:
# Example usage with your data, looking for pairs with '1' in one and '4' in the other
pair1, pair2 = get_pairs_with_one_difference_and_specific_digits(data, '1', '5')
print("Pair 1:", pair1)
print("Pair 2:", pair2)

Pair 1: {'pair': [('0125', 2), ('0145', 1)], 'difference': {'1', '5'}}
Pair 2: {'pair': [('01345', 2), ('02345', 1)], 'difference': {'1', '5'}}


In [ ]:
get_pairs_with_one_difference

Recall $f_{\sigma_{\Gamma}}$ as a dictionary with keys recording the elementary induced subgraphs $(\Gamma^{'},\Gamma^{'c})$ with values recording $\alpha_{\sigma_{\Gamma}}(\Gamma^{'}),\alpha_{\sigma_{\Gamma}}(\Gamma^{'c}),$ the number of edgess in the cut of $(\Gamma^{'},\Gamma^{'c})$ and a True or False statement if the pair satisfies,

$$\alpha_{\sigma_{\Gamma}}(\Gamma^{'})+\alpha_{\sigma_{\Gamma}}(\Gamma^{'c}) = d- |\#E(\Gamma) \setminus (E(\Gamma^{'} \cup E(\Gamma^{'c})|+1.$$